In [17]:
import pandas as pd 
import duckdb 

In [21]:
#requete 
def requetage(source = 'source_donnees', requete = 'demande') :
 conduck = duckdb.connect(database=':memory:', read_only=False)
 source_duck = conduck.read_parquet(source)
 requete_donnees = requete 
 pl_resultat = conduck.execute(requete_donnees).df()
 conduck.close()
 return pl_resultat


conduck = duckdb.connect(database=':memory:', read_only=False)
source_siren = conduck.read_parquet('../../data/infos_siren_siret_cog_collectivites.parquet')
transco = pd.read_parquet('./parquet/transcodage_2020')


In [25]:

df_rouen_agglo = requetage(source='../exploration_viz/db_2020.parquet', 
    requete = '''

    Select 

    a.Id_Fichier, b.denominationUniteLegale, b.libelleCategorieJuridique, b.libelleCommuneEtablissement, 
       a.DteStr, a.LibellePoste, a.TypOpBudg,
       a.OpeCpteTiers, a.NatCEPL, a.Departement, a.NatDec, a.Exer,
       a.ProjetBudget, a.NatVote, a.OpeEquip, a.VoteFormelChap, a.TypProv,
       a.BudgPrec, a.ReprRes, a.NatFonc, a.CodTypBud, a.LibelleEtabPal,
       a.IdEtabPal, a.LibelleEtab, a.IdEtab, a.CodColl, a.CodInseeColl,
       a.CodBud, a.Nomenclature, a.Nature, a.Fonction, a.LibCpte, a.ContNat,
       a.ArtSpe, a.ContFon, a.CodRD, a.MtBudgPrec, a.MtRARPrec, a.MtPropNouv,
       a.MtPrev, a.CredOuv, a.MtReal, a.MtRAR3112, a.OpBudg, 
       a.IdColl, a.IdPost, b.siren, b.siret,  b.complementAdresseEtablissement
    
    from source_duck as a 
    left join source_siren as b   
     ON a.IdColl = b.siret
       
       WHERE a.NatDec = '09' 
        AND b.REG = '28'
        AND a.OpBudg = '0'
        AND b.libelleCommuneEtablissement ='ROUEN'    
    ''')

df_rouen_agglo['MtReal'] = df_rouen_agglo['MtReal'].astype(float).round(2)
df_rouen_agglo['CredOuv'] = df_rouen_agglo['CredOuv'].astype(float).round(2)
df_rouen_agglo['Exer'] = df_rouen_agglo['Exer'].astype(int)
df_rouen_agglo = df_rouen_agglo.drop_duplicates()

In [20]:
df_transco = pd.read_parquet('./parquet/transcodage_2020')
df_transco.head()

,Code,Lib_court,Libelle,PourEtatSeul,Section,Special,TypeChapitre,Categorie,DEquip,DOES,...,DR,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis,Nomenclature,Annee
0,002,Résultat de fonctionnement reporté,Résultat de fonctionnement reporté,None,F,0,L,Nature,None,None,...,None,None,None,None,None,None,None,None,M14-M14_COM_500_3500,2020
1,011,Charges à caractère général,Charges à caractère général,None,F,0,G,Nature,None,None,...,None,None,None,None,None,None,None,None,M14-M14_COM_500_3500,2020
2,012,"Charges de personnel, frais assimilés",Charges de personnel et frais assimilés,None,F,0,G,Nature,None,None,...,None,None,None,None,None,None,None,None,M14-M14_COM_500_3500,2020
3,013,Atténuations de charges,Atténuations de charges,None,F,0,G,Nature,None,None,...,None,None,None,None,None,None,None,None,M14-M14_COM_500_3500,2020
4,014,Atténuations de produits,Atténuations de produits,None,F,0,G,Nature,None,None,...,None,None,None,None,None,None,None,None,M14-M14_COM_500_3500,2020


In [27]:
pd.set_option('display.max_columns', None)
transco[['Code', 'Lib_court', 'Libelle', 'Categorie', 'Nomenclature']]

,Code,Lib_court,Libelle,Categorie,Nomenclature
0,002,Résultat de fonctionnement reporté,Résultat de fonctionnement reporté,Nature,M14-M14_COM_500_3500
1,011,Charges à caractère général,Charges à caractère général,Nature,M14-M14_COM_500_3500
2,012,"Charges de personnel, frais assimilés",Charges de personnel et frais assimilés,Nature,M14-M14_COM_500_3500
3,013,Atténuations de charges,Atténuations de charges,Nature,M14-M14_COM_500_3500
4,014,Atténuations de produits,Atténuations de produits,Nature,M14-M14_COM_500_3500
...,...,...,...,...,...
888,802,Engagements reçus,Engagements reçus,Nature_compte,M14-M14_CE
889,86,Valeurs inactives,Valeurs inactives,Nature_compte,M14-M14_CE
890,861,"Comptes posit° : titres, valeurs portef.",Comptes de position : titres et valeurs en por...,Nature_compte,M14-M14_CE
891,862,"Comptes posit° : titres, valeurs corresp",Comptes de position : titres et valeurs chez l...,Nature_compte,M14-M14_CE


In [32]:
transco['Categorie'].drop_duplicates()

0                Nature
47        Nature_compte
1054       Fonction_Ref
1760           Fonction
1816    Fonction_compte
Name: Categorie, dtype: object

In [33]:
transco[(transco['Categorie'] == 'Fonction_Ref')]

,Code,Lib_court,Libelle,PourEtatSeul,Section,Special,TypeChapitre,Categorie,DEquip,DOES,DOIS,DR,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis,Nomenclature,Annee
1054,0,Services généraux,Services généraux,None,None,None,None,Fonction_Ref,None,None,None,None,None,None,None,None,None,None,None,M14-M14_CCAS_SUP3500,2020
1055,01,Opérations non ventilables,Opérations non ventilables,None,None,None,None,Fonction_Ref,None,None,None,None,None,None,None,None,None,None,None,M14-M14_CCAS_SUP3500,2020
1056,02,Administration générale du C.C.A.S.,Administration générale du C.C.A.S.,None,None,None,None,Fonction_Ref,None,None,None,None,None,None,None,None,None,None,None,M14-M14_CCAS_SUP3500,2020
1057,5,Interventions sociales,Interventions sociales,None,None,None,None,Fonction_Ref,None,None,None,None,None,None,None,None,None,None,None,M14-M14_CCAS_SUP3500,2020
1058,52,Interventions sociales,Interventions sociales,None,None,None,None,Fonction_Ref,None,None,None,None,None,None,None,None,None,None,None,M14-M14_CCAS_SUP3500,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,92,Aides à l'agriculture et aux industries,Aides à l'agriculture et aux industries agro-a...,None,None,None,None,Fonction_Ref,None,None,None,None,None,None,None,None,None,None,None,M14-M14_COM_SUP3500,2020
2195,93,"Aides à l'énergie, indus. manufact., BTP","Aides à l'énergie, aux industries manufacturiè...",None,None,None,None,Fonction_Ref,None,None,None,None,None,None,None,None,None,None,None,M14-M14_COM_SUP3500,2020
2196,94,Aides commerce et services marchands,Aides au commerce et aux services marchands,None,None,None,None,Fonction_Ref,None,None,None,None,None,None,None,None,None,None,None,M14-M14_COM_SUP3500,2020
2197,95,Aides au tourisme,Aides au tourisme,None,None,None,None,Fonction_Ref,None,None,None,None,None,None,None,None,None,None,None,M14-M14_COM_SUP3500,2020


In [37]:
df = df_rouen_agglo[['Nomenclature', 'Nature', 'ContNat', 'Fonction']]

In [84]:
transco1 = transco[['Code', 'Lib_court', 'Categorie', 'Nomenclature']]
contnat = transco1[transco1['Categorie'] == 'Nature'][['Code', 'Lib_court', 'Nomenclature']].rename(columns = {
    'Code': 'ContNat', 'Lib_court' : 'ConNat_Lib'})

nature = transco1[transco1['Categorie'] == 'Nature_compte'][['Code', 'Lib_court', 'Nomenclature']].rename(columns = {
    'Code': 'Nature', 'Lib_court' : 'Nature_Lib'})

fonction = transco1[transco1['Categorie'] == 'Fonction_Ref'][['Code', 'Lib_court', 'Nomenclature']].rename(columns = {
    'Code': 'Fonction', 'Lib_court' : 'Lib_Fonction'})
contnat.head(1)

,ContNat,ConNat_Lib,Nomenclature
0,002,Résultat de fonctionnement reporté,M14-M14_COM_500_3500


In [83]:
df1 = df.merge(nature, on = ['Nomenclature', 'Nature'], how = 'left')
df1 = df1.merge(ContNat, on = ['Nomenclature', 'ContNat'], how = 'left')
df1 = df1.merge(fonction, on = ['Nomenclature', 'Fonction'], how = 'left')
df1 

,Nomenclature,Nature,ContNat,Fonction,Nature_Lib,ConNat_Lib,Lib_Fonction
0,M52-M52,6135,011,None,Locations mobilières,Charges à caractère général,NaN
1,M52-M52,6132,011,None,Locations immobilières,Charges à caractère général,NaN
2,M52-M52,62872,011,None,Remb. frais au budget annexe,Charges à caractère général,NaN
3,M52-M52,64113,012,None,NBI,Charges de personnel et frais assimilés,NaN
4,M52-M52,6417,012,None,Rémunérations des apprentis,Charges de personnel et frais assimilés,NaN
...,...,...,...,...,...,...,...
4909,M14-M14_COM_SUP3500,4581,4581,01,Opérations investissement sous mandat,Opérations sous mandat,Opérations non ventilables
4910,M14-M14_COM_SUP3500,4582,4582,01,Opérations investissement sous mandat,Opérations sous mandat,Opérations non ventilables
4911,M14-M14_COM_SUP3500,165,16,71,Dépôts et cautionnements reçus,Emprunts et dettes assimilées,Parc privé de la ville
4912,M14-M14_COM_SUP3500,4581,4581,01,Opérations investissement sous mandat,Opérations sous mandat,Opérations non ventilables
